In [64]:
import pandas as pd
#init random seed
import numpy as np
from sklearn.model_selection import train_test_split
from matrix_factorization_with_als import MF
import mf_als
from sklearn.metrics import mean_squared_error, mean_absolute_error
from imp import reload
reload(mf_als)

<module 'mf_als' from 'C:\\Users\\irina\\Dev\\master\\mf_als.py'>

In [65]:
df = pd.read_json(r'D:\Datasets\amazon_reviews\processed\reviews_Toys_and_Games_5.json')

In [66]:
df_train, df_test = train_test_split(df, test_size = 0.3, stratify=df['reviewerID'], random_state=42)

In [67]:
pivoted = df_train.pivot(index = 'reviewerID', columns = 'asin', values = 'overall')
pivoted = pivoted.fillna(0.)

In [68]:
rating_matrix = pivoted.values

In [69]:
mf = mf_als.MF(k=15, rating_matrix=rating_matrix, lambda_u=0.01, lambda_v=0.01)

In [70]:
for e in range(200):
    U , V, beta_u, beta_v, err = mf.ALS()
    print('Epoch ', e, ' error ', err)
    
    if e % 5 == 0:
        test()

Epoch  0  error  4.11723306607147
MSE: 1.5175759578683083
MAE: 1.1223835490811394
Epoch  1  error  4.080536321675118
Epoch  2  error  4.060242996077528
Epoch  3  error  4.046898040138394
Epoch  4  error  4.0367987437324055
Epoch  5  error  4.028512115878082
MSE: 1.357399358683545
MAE: 1.0325139620266515
Epoch  6  error  4.021301945556858
Epoch  7  error  4.014845710302877
Epoch  8  error  4.008942131827233
Epoch  9  error  4.0034867839691515
Epoch  10  error  3.998371935896285
MSE: 1.3314376641181118
MAE: 1.0178810101975415
Epoch  11  error  3.99355636255975
Epoch  12  error  3.9889661588685805
Epoch  13  error  3.984590875030638
Epoch  14  error  3.980400208633726
Epoch  15  error  3.976381006874041
MSE: 1.3142085303953326
MAE: 1.0082200206749934
Epoch  16  error  3.9725156014150116
Epoch  17  error  3.9687925128114507
Epoch  18  error  3.965199652165322
Epoch  19  error  3.961728652153048
Epoch  20  error  3.9583715100745835
MSE: 1.3014679913493472
MAE: 1.0011349734592805
Epoch  21  

MemoryError: 

In [51]:
predictions = np.mean(rating_matrix) +  np.dot(U, V.T) + beta_u.reshape(-1, 1) + beta_v

In [52]:
# train set
print("MSE (non zero): %s" % mean_squared_error(rating_matrix[rating_matrix > 0], predictions[rating_matrix > 0]) ** 0.5)
print("MAE (non zero): %s" % mean_absolute_error(rating_matrix[rating_matrix > 0], predictions[rating_matrix > 0]))

MSE (non zero): 0.10411304602241836
MAE (non zero): 3.9231425063914873


In [29]:
# test set
def test():
    predictions = np.mean(rating_matrix) + np.dot(U, V.T) + beta_v + beta_u.reshape(-1, 1)
    preds_df_unmelt = pd.DataFrame(predictions, columns = pivoted.columns, index = pivoted.index)
    preds_df_unmelt.index.name = 'reviewerID'
    preds_df_unmelt.columns.name = 'asin'
    df_test_val = df_test.copy()
    df_test_val['value'] = 0

    def get_val(x):
        if x['reviewerID'] in preds_df_unmelt.index:
            if x['asin'] in preds_df_unmelt.columns:
                return preds_df_unmelt.loc[x['reviewerID'], x['asin']]
        return None

    df_test_val['value'] = df_test_val.apply(get_val, axis = 1)

    mse = mean_squared_error(df_test_val[~df_test_val.value.isnull()].overall, df_test_val[~df_test_val.value.isnull()].value) ** 0.5
    mae = mean_absolute_error(df_test_val[~df_test_val.value.isnull()].overall, df_test_val[~df_test_val.value.isnull()].value)

    print("MSE: %s" % mse)
    print("MAE: %s" % mae)

In [ ]:
# for lambdas 0.1 => MSE: 1.4
    